In [ ]:
# Prepare a CNN Model for prediction of Lung Cancer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import libraries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from zipfile import ZipFile

In [ ]:
data_path = '/content/drive/MyDrive/lung-cancer-dataset.zip'

from zipfile import ZipFile
with ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')
    print("Extraction Done")


Extraction Done


In [ ]:
# import zipfile

# data_path = "lung-cancer-dataset.zip"

# try:
#     with zipfile.ZipFile(data_path, 'r') as zip_ref:
#         zip_ref.extractall("unzipped_content")
#         print("Extraction Done")
# except zipfile.BadZipFile as e:
#     print("Bad zip file:", e)
# except Exception as e:
#     print("Some other error:", e)


Bad zip file: File is not a zip file


In [ ]:
# set the image size and path
image_size =256
batch_size=32
df="/content/dataset/lung-cancer-dataset/lung_images"

In [ ]:
#pre-processing our Imgae(Downscalling the image)
datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2
)

In [ ]:
train_generator = datagen.flow_from_directory(
    df,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode= 'categorical',
    subset='training',
    shuffle=True
)

# prepare our training dataset
# train_generator = datagen.flow_from_directory(
#     df,
#     target_size=(image_size, image_size),
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='training',
#     shuffle=True
# )

Found 2402 images belonging to 3 classes.


In [ ]:
# prepare our validation dataset
val_generator = datagen.flow_from_directory(
    df,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 600 images belonging to 3 classes.


In [ ]:
# prepare the convolutional model
model = Sequential([
    Conv2D(32,(3,3),activation="relu",input_shape=(image_size,image_size,3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128,(3,3),activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),# it will flatten the matrix so that i can pass my input to hidden layer
    Dense(128,activation="relu"),# hidden layer
    Dropout(0.5),
    Dense(3,activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy",])

In [ ]:
# check the summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,363 (56.61 MB)

 Trainable params: 14,839,363 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#earlystopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
model.save('lung_cancer_classifier.h5')
